## LSTM

In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
from headers import *
from spacy.vocab import Vocab
from spacy.language import Language
import spacy



In [2]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.layers import BatchNormalization
from keras.layers.recurrent import LSTM
from keras import optimizers
import tensorflow as tf

Using TensorFlow backend.


In [ ]:
%run 'utils.ipynb'



In [ ]:
#train_dict,test_dict=train_test_split(dict2,SPLIT_RATIO)
#print len(train_dict),len(test_dict)

#X_train,X_val,X_test,Y_train,Y_val,Y_test=X_set[:-2000],X_set[-2000:-1000],X_set[-1000:],Y_set[:-2000],Y_set[-2000:-1000],Y_set[-1000:]
#print X_train.shape,X_val.shape,X_test.shape,Y_train.shape,Y_val.shape,Y_test.shape

In [3]:

model = Sequential()
model.add(LSTM(128, return_sequences=True, input_shape=(DEF_LEN, WV_SIZE)))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(LSTM(128, return_sequences=False))
model.add(BatchNormalization())
model.add(Dense(WV_SIZE, kernel_initializer='glorot_normal'))
model.add(Activation('softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 10, 128)           117248    
_________________________________________________________________
batch_normalization_1 (Batch (None, 10, 128)           512       
_________________________________________________________________
dropout_1 (Dropout)          (None, 10, 128)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               131584    
_________________________________________________________________
batch_normalization_2 (Batch (None, 128)               512       
_________________________________________________________________
dense_1 (Dense)              (None, 100)               12900     
_________________________________________________________________
activation_1 (Activation)    (None, 100)               0         
Total para

In [4]:
from keras.utils.vis_utils import plot_model
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
plot_model(model, to_file='lstm.png', show_shapes=True)

In [ ]:
#rprop=optimizers.RMSprop(lr=0.002)
adam=optimizers.Adam(lr=3e-3)

model.compile(loss=sq_distance, optimizer=adam)


In [ ]:
model.fit(X_train[:30000],Y_train[:30000],batch_size=100,epochs=5,validation_data=(X_val,Y_val),verbose = 1)

In [ ]:
model.evaluate(x=X_test,y=Y_test,batch_size=100, verbose=1)

In [ ]:
X_train[0,:,:]

In [ ]:
a=model.predict(X_test[45,:,:].reshape(1,DEF_LEN,WV_SIZE), batch_size=1, verbose=1)

In [ ]:
#get_most_similar(a)
dict2[0][1]


In [ ]:
get_most_similar(a)[:5]

In [ ]:
a


In [ ]:
dict2[0]
print X_test.shape

In [ ]:
def get_most_similar_for_lstm(input_embedding):
    cs_list=[]
    for v in global_vocab:
        doc=nlp(v)
        #if doc[0].has_vector:
            #if doc[0].vector.all()!=0.0:
        cs=cosine_sim(input_embedding,doc[0].vector)
                #print cs
                #print len(input_embedding),len(nlp.vocab.get_vector(unicode(v,'utf-8')))
                #if not math.isnan(cs):
        cs_list.append((doc[0].text,cs))
            
                
                
                    
    sorted_cs_list=sorted(cs_list,key=lambda tup:tup[1],reverse=True)
    #return sorted_cs_list[:return_length]
    return sorted_cs_list

In [ ]:
sample=dict2[-100:]
#print sample
#random.shuffle(sample)
#print sample
test_words=[x for x,y in sample]
ranks=[]
X_test_embed=np.zeros((100,DEF_LEN,WV_SIZE))
X_test_embed[:,:,:]=X_test[-100:,:,:]

predictions=model.predict(X_test[-100:,:,:],batch_size=10,verbose=0)
print "Done"


        
    
    



In [ ]:
for i in range(len(predictions)):
    input_embedding=predictions[i]
    
    r=get_list_from_tuple(get_most_similar(input_embedding))
    results=r[0]
    #print len(results)==len(local_vocab)
    scores=r[1]
    #top_recall_results=results[:recall_count]
    #target_rank=results.index(target_word)
    #if target_word[0].text in results:
        #print target_word[0].text
        #print "There"'
    target_word=Y_GT[-100+i]
    print dict2[-100+i][1],' ',target_word,'/',results[0],' ',scores[0]
    #print results.index(target_word)
    if target_word in results:
        target_rank=results.index(target_word)
        ranks.append(target_rank)
        print "%d/%d done for word %s, rank: %d"%(i,len(test_words),str(target_word),target_rank)
    #print "Final accuracy on %d test points: %f"%(len(test_set),(float(acc)/float(len(test_set))))
print "Mean rank: %f"%(float(sum(ranks))/float(len(ranks)))
print get_rank_bins(ranks)

In [ ]:
'''%Sentence that could be used:
%structure on which food and drinks are kept: counter
%way of dealing with a difficult situation: solution
%make someone able to do something: prepare
%affecting a large area:extensive
%expressing lack of interest: levity
%not able to handle things: clumsy
%tendency to think negative: pessimism
%thing to keep clothes on: hanger
%symbol behind a card: trademark
%bottle that you push: dispenser'''

In [ ]:
dict2.append(('counter','structure on which food and drinks are kept'))

In [ ]:
dict2.append(('solution','way of dealing with a difficult situation'))

In [ ]:
dict2.append(('prepare','make someone able to do something'))

In [ ]:
dict2.append(('extensive','affecting a large area'))

In [ ]:
dict2.append(('levity','expressing lack of interest'))

In [ ]:
dict2[-5:]

In [ ]:
sample=dict2[-5:]
test_em=np.zeros((5,DEF_LEN,WV_SIZE))
for i in range(len(sample)):
    
    tok=sample[i][1].split(' ')
    for t in range(len(tok)):
        test_em[i,t,:]=nlp(unicode(tok[t],'utf-8')).vector
print test_em.shape
    

In [ ]:
predictions=model.predict(test_em,batch_size=2)

In [ ]:
for i in range(len(predictions)):
    print i,get_most_similar(predictions[i])[:5]

In [ ]:
print eval_method_with_spacy(dict2[-5:],5000,method='additive')